# Olá Mundo Concorrente

## Como animar texto na saída padrão

Imagine um **teletipo** ([teletype](http://www.columbia.edu/cu/computinghistory/teletype/index.html)): uma máquina de escrever automatizada que pode trocar mensagens via modem ou comunicação serial com outro teletipo ou com um computador.

In [1]:
import time

def relogio():
    while True:
        print('\r' + time.strftime('%H:%M:%S'), end='')
        time.sleep(1)

In [2]:
# relogio()

In [3]:
def girar():
    for char in '|/-\\':
        print(f'\r{char}', end=' ')
        time.sleep(1)

In [4]:
# girar()

In [5]:
import itertools

def girar():
    for char in itertools.cycle('|/-\\'):
        print(f'\r{char}', end=' ', flush=True)
        time.sleep(.1)

In [6]:
# girar()

In [7]:
from unicodedata import lookup
DOTS = (1, 2, 3, 7, 8, 6, 5, 4)
PREFIX = 'BRAILLE PATTERN DOTS-'

for dot in DOTS:
    print(lookup(f'{PREFIX}{dot}'), end='')

⠁⠂⠄⡀⢀⠠⠐⠈

## Exemplo sequencial

In [8]:
def girar(msg):
    for char in itertools.cycle(reversed('⠁⠂⠄⡀⢀⠠⠐⠈')):
        print(f'\r{char} {msg}', end=' ', flush=True)
        time.sleep(.1)

def main():
    girar('pensando para sempre...')

In [9]:
#main()

## Exemplos com threads

Duas threads: a thread principal e aquela nós criamos explicitamente.

In [10]:
import itertools
import time
from threading import Thread, Event

def girar(msg: str, calculado: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if calculado.wait(0.05):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')


def buscar() -> int:
    time.sleep(3)
    return 42


def main():
    pronto = Event()
    fio = Thread(target=girar, args=['buscando...', pronto])
    fio.start()
    res = buscar()
    pronto.set()
    fio.join()
    print(res)


In [11]:
# main()

## Exemplos com processos ou corrotinas

As versões com processos (`multiprocessing`) ou corrotinas (`asyncio`) não funcionam aqui por diferentes motivos ligados à
[arquitetura do no Jupyter Notebook](https://docs.jupyter.org/en/latest/projects/architecture/content-architecture.html#the-jupyter-notebook-interface).

<img src="notebook_components.png" width="50%">

**DICA**: Rode no terminal os programas `gira_proc.py` e `gira_async`.

## Exemplo com curio

**[curio](https://github.com/dabeaz/curio/)** é uma biblioteca de programação assíncrona experimental do professor David Beazley que apresentou novas formas de usar as instruções `async for` e `async wait`, de uma forma mais *pythonica* do que a própria biblioteca `asyncio` (que foi criada antes das instruções `async`).

Este exemplo funciona, mas só usando a versão mais recente do
[repositório do curio](https://github.com/dabeaz/curio/) e não o pacote `curio` do PyPI.

In [17]:
#!/usr/bin/env python3
from curio import run, TaskGroup
import curio.socket as socket
from keyword import kwlist

MAX_KEYWORD_LEN = 5
NAMES = [kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN]

async def probe(domain: str) -> tuple[str, bool]:  # (1)
    try:
        await socket.getaddrinfo(domain, None)  # (2)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)

async def main() -> None:
    domains = (f'{name}.dev'.lower() for name in NAMES)
    async with TaskGroup() as group:  # (3)
        for domain in domains:
            await group.spawn(probe, domain)  # (4)
        async for task in group:  # (5)
            domain, found = task.result
            mark = '✓' if found else ' '
            print(f'{mark} {domain}')

if __name__ == '__main__':
    from time import perf_counter
    t0 = perf_counter()
    run(main())  # (6)
    dt = perf_counter() - t0
    print(f'{len(NAMES)} domains probed in {dt:0.4f}s')

✓ false.dev
  none.dev
  true.dev
✓ await.dev
✓ del.dev
✓ from.dev
✓ and.dev
✓ def.dev
✓ async.dev
  break.dev
  or.dev
  class.dev
  is.dev
  elif.dev
  else.dev
✓ in.dev
  pass.dev
  for.dev
✓ as.dev
  if.dev
  with.dev
✓ not.dev
✓ raise.dev
✓ try.dev
  yield.dev
✓ while.dev
26 domains probed in 0.0146s
